In [1]:
%%bash
if [ ! -f ./train.csv ]; then
    kaggle competitions download -c titanic
    unzip titanic.zip
fi

In [2]:
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)

Note: you may need to restart the kernel to use updated packages.


In [49]:
train_df = pd.read_csv("./train.csv") #import train dataset
test_df = pd.read_csv("./test.csv")   #import test dataset
print(train_df.shape)                 #in ra kích thước của train dataset
train_df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
train_df["Survived"].mean()            #tính trung bình của cột Survived

0.3838383838383838

In [51]:
train_df.isnull().sum()                # hàm isnull() trả về True tại các ô không có giá trị, tính sum tại các ô đấy

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [52]:
from sklearn import preprocessing

In [53]:
onehot = preprocessing.OneHotEncoder(drop="first") 
le = preprocessing.LabelEncoder()

In [54]:
drop_cols = ["Name", "Age", "Ticket", "Cabin", "Embarked", "PassengerId"]
train_df.drop(drop_cols, axis=1, inplace=True)
test_df.drop(drop_cols, axis=1, inplace=True)

In [55]:
train_df["Sex"] = le.fit_transform(train_df["Sex"])
test_df["Sex"] = le.transform(test_df["Sex"])

In [56]:
print(test_df["Sex"])
print(train_df["Sex"])

0      1
1      0
2      1
3      1
4      0
      ..
413    1
414    0
415    1
416    1
417    1
Name: Sex, Length: 418, dtype: int32
0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int32


In [57]:
train_df.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare
0,0,3,1,1,0,7.2500
1,1,1,0,1,0,71.2833
2,1,3,0,0,0,7.9250
3,1,1,0,1,0,53.1000
4,0,3,1,0,0,8.0500


In [59]:
train_df["Pclass"].values[:,None].shape   #Sử dụng None là trick để thêm 1 chiều vào dataframe

(891,)

In [61]:
onehot_vals = onehot.fit_transform(train_df["Pclass"].values[:,None]).todense()
onehot_vals

matrix([[0., 1.],
        [0., 0.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 0.],
        [0., 1.]])

In [43]:
import pdb

onehot_vals = onehot.fit_transform(train_df["Pclass"].values[:,None]).todense()   #fit cột Pclass và dạng ont-hot, dùng todense dể biến dữ liệu sau khi encode thành dạng ma trận
class_names = onehot.categories_[0][1:]                          #lấy n-1 categories
pclass = pd.DataFrame(onehot_vals, columns=class_names)          #tạo dataframe với các giá trị onehot_vals, tên các cột lấy từ class_name
train_df.drop("Pclass", axis=1, inplace=True)                    #từ tập train_df xóa bỏ luôn cột Plass do sử dụng inplace=True
train_df = pd.concat([train_df, pclass], axis=1)                 #concatenate train_df với pclass và gán cho train_df
# pdb.set_trace()

onehot_vals = onehot.transform(test_df["Pclass"].values[:, None]).todense()  #tương tự với test_df
pclass = pd.DataFrame(onehot_vals, columns=class_names)
test_df.drop("Pclass", axis=1, inplace=True)
test_df = pd.concat([test_df, pclass], axis=1)

In [44]:
train_df.head()

,Survived,Sex,SibSp,Parch,Fare,2,3
0,0,1,1,0,7.2500,0.0,1.0
1,1,0,1,0,71.2833,0.0,0.0
2,1,0,0,0,7.9250,0.0,1.0
3,1,0,1,0,53.1000,0.0,0.0
4,0,1,0,0,8.0500,0.0,1.0


In [70]:
onehot_vals

matrix([[0., 1.],
        [0., 0.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 0.],
        [0., 1.]])

In [69]:
pclass

,2,3
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
413,0.0,1.0
414,0.0,0.0
415,0.0,1.0
416,0.0,1.0
